# 03 - Langchain

In this lab, we will introduce [Langchain](https://python.langchain.com/docs/get_started/introduction), a framework for developing applications powered by language models.

Langchain supports Python and Javascript / Typescript. For this lab, we will use Python.

We'll start by importing the `AzureOpenAI` specific components from the `langchain` package, including models and schemas for interacting with the API.

In [1]:
from langchain_openai import AzureChatOpenAI

As with all the other labs, we'll need to provide our API key and endpoint details, so we'll load them from our `.env` file.

In [2]:
import os
import openai
from dotenv import load_dotenv

if load_dotenv():
    print("Found Azure OpenAI Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("No file .env found")

Found Azure OpenAI Endpoint: https://aoai-itia-ws.openai.azure.com/


Next, we'll configure Langchain by providing the Azure OpenAI deployment name. Langchain will automatically retrieve details for the Azure OpenAI endpoint and version from the environment variables we've set above.

In [3]:
# Create an instance of Azure OpenAI
llm = AzureChatOpenAI(
    azure_deployment = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
)

## Send a prompt to Azure OpenAI using Langchain

We're now ready to send a request to Azure OpenAI. To do this, we invoke the `llm` instance we created above and pass in the prompt.

In [6]:
r = llm.invoke("What things could I make with a Raspberry Pi?")

# Print the response
print(r.content)

There are numerous projects you can undertake using a Raspberry Pi. Here are some popular examples:

1. Media center: Transform your Pi into a media center by installing Kodi or Plex, allowing you to stream movies, TV shows, and music to your TV.

2. Retro gaming console: Build a retro gaming console using the open-source RetroPie software, enabling you to play classic games from various consoles and arcade systems.

3. Home automation system: Automate your home using a Raspberry Pi by connecting and controlling various smart devices such as lights, thermostats, and security systems.

4. Network-attached storage (NAS): Set up a NAS using a Raspberry Pi, connecting external hard drives and creating a centralized storage solution for your home network.

5. Weather station: Build a weather station that collects and displays data such as temperature, humidity, and air pressure using sensors connected to your Raspberry Pi.

6. Robotics projects: Use a Raspberry Pi to control and program rob

Compared to using the OpenAI Python library as we did in the previous lab, Langchain further simplified the process of interacting with the LLM by reducing it to a `llm.invoke` call.

## Using templates and chains

We've seen that we can use Langchain to interact with the LLM and it's a little easier to work with than the OpenAI Python library. However, that's just the start of how Langchain makes it easier to work with LLM's. Most OpenAI models are designed to be interacted with using a Chat style interface, where you provide a persona or system prompt which helps the LLM understand the context of the conversation. This will then be sent to the LLM along with the user's request.

So that you don't have to setup the persona / system prompt every time you want to interact with the LLM, Langchain provides the concept of Templates. Templates are a way to define the persona and system prompt once and then reuse them across multiple interactions.

In [7]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a chatbot that helps people generate ideas for their next project. You can help them brainstorm ideas, come up with a plan, or even help them with their project."),
    ("user", "{input}")
])

Above we've defined a "system" message which will tell the LLM how we're expecting it to respond, and an `{input}` placeholder for the user's prompt.

Next, we define a chain. A chain allows us to define a sequence of operations that we want to perform. In this case, we're defining a simple chain that will take the prompt we've defined above and send it to the LLM.

In [8]:
chain = prompt | llm

Now, we can invoke the chain in a similar fashion to how to invoked the LLM earlier. This time, we're passing in the user's input as a parameter to the chain, which will replace the `{input}` placeholder in the prompt.

In [9]:
chain.invoke({"input": "I've just purchased a Raspberry Pi and I'm looking for a project to work on. Can you help me brainstorm some ideas?"})

AIMessage(content='Of course! Here are a few project ideas you can consider for your Raspberry Pi:\n\n1. Retro gaming console: Turn your Raspberry Pi into a retro gaming console by installing RetroPie. You can play classic games from various consoles, such as NES, SNES, Sega Genesis, and more.\n\n2. Home media center: Transform your Raspberry Pi into a media center by setting up Kodi. You can stream movies, TV shows, music, and even access online content through various add-ons.\n\n3. Weather station: Build a weather station using your Raspberry Pi to monitor and display real-time weather data from your location. You can include temperature, humidity, barometric pressure, and more.\n\n4. Home automation hub: Create a home automation system by connecting your Raspberry Pi to various devices like smart plugs, lights, sensors, and more. Control and automate your home using software like Home Assistant or OpenHAB.\n\n5. Pi-powered robot: Build a small robot using your Raspberry Pi and cont

The result will be an `AIMessage` object, which contains the response from the LLM.

Let's enhance the chain further to get it to parse the output from the LLM and extract the text from the response. First, we define an output parser.

In [10]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Next, we redefine our chain to include the output parser. So now when we invoke the chain, it will 

- Take the prompt template and add the user's input
- Send the prompt to the LLM
- Parse the response from the LLM and extract the text

In [11]:
chain = prompt | llm | output_parser

Now let's invoke the chain again with the same prompt as before.

In [12]:
chain.invoke({"input": "I've just purchased a Raspberry Pi and I'm looking for a project to work on. Can you help me brainstorm some ideas?"})

"Of course! Raspberry Pi is a versatile device that can be used for a wide range of projects. Here are a few ideas to get you started:\n\n1. Retro gaming console: Turn your Raspberry Pi into a retro gaming console using software like RetroPie. Connect it to a TV, add some game controllers, and enjoy playing classic games.\n\n2. Home media center: Transform your Raspberry Pi into a media center by installing Kodi. You can stream movies, TV shows, music, and even access online content, all from the comfort of your living room.\n\n3. Smart home automation: Build a smart home system using your Raspberry Pi. You can control lights, appliances, and other devices using sensors and relays. Add voice control with a microphone and speaker.\n\n4. Weather station: Create a weather station by connecting sensors to your Raspberry Pi. Monitor temperature, humidity, air pressure, and more. Display the data on an LCD screen or access it remotely through a web interface.\n\n5. Home security system: Deve

This time, you should only get a string containing the text from the response.

We can do much more powerful things with chains than simply setting up and passing prompts to the LLM and parsing the results. We can augment the prompt with external data retrieved from a database, we could add conversation history to provide context for a chatbot, or we could even chain multiple LLMs together to create a more powerful model. We'll explore some of these ideas in future labs.

## Summary

Langchain is an example of an AI orchestrator. It provides an alternative method to the raw API or using an SDK package to access the AI models, but on top of that can provide additional integrations, deal with issues related to rate limiting of the API and provide an abstraction layer over potentially complex operations. We'll get into those more complex use cases in later labs.

## Up Next

In the next lab, we'll look at another AI Orchestrator - Semantic Kernel.

## Next Section

📣 [Semantic Kernel](../04-SemanticKernel/semantickernel.ipynb)